# ROTATE, CROP, SYMETRY TEST

## rotate the mask around the longest diameter

In [2]:

import cv2
import numpy as np

def find_longest_diameter(mask_image):
    # Find contours of the lesion in the mask
    contours, _ = cv2.findContours(mask_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        print("Error: No contour found in the mask image.")
        return None
    
    # Find the convex hull of the largest contour
    largest_contour = max(contours, key=cv2.contourArea)
    hull = cv2.convexHull(largest_contour)
    
    # Find the longest diameter within the convex hull
    longest_diameter = find_longest_line(hull)
    
    return longest_diameter

def find_longest_line(contour):
    # Initialize variables to store the longest line and its length
    max_length = 0
    longest_line = None
    
    # Iterate through all combinations of points to find the longest line segment
    for i in range(len(contour)):
        for j in range(i + 1, len(contour)):
            # Calculate the length of the line segment
            length = np.linalg.norm(contour[i] - contour[j])
            
            # Update the longest line if the current line segment is longer
            if length > max_length:
                max_length = length
                longest_line = (contour[i][0], contour[j][0])
    
    return longest_line

def rotate_image(image, angle):
    # Get the dimensions of the image
    h, w = image.shape[:2]
    
    # Calculate the rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D((w/2, h/2), angle, 1)
    
    # Perform the rotation
    rotated_image = cv2.warpAffine(image, rotation_matrix, (w, h))
    
    return rotated_image

def crop_to_sides(mask_image, longest_diameter):
    # Determine the angle of rotation to make the longest diameter vertical
    angle = np.degrees(np.arctan2(longest_diameter[1][1] - longest_diameter[0][1], longest_diameter[1][0] - longest_diameter[0][0]))
    
    # If the angle is more than 90 degrees, subtract 180 degrees to make it negative
    if angle > 90:
        angle -= 180
    
    # Rotate the mask image
    rotated_mask = rotate_image(mask_image, angle)
    
    # Find contours of the rotated lesion
    contours, _ = cv2.findContours(rotated_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Find the bounding rectangle of the rotated lesion
    x, y, w, h = cv2.boundingRect(contours[0])
    
    # Crop the rotated mask image to include the left and right sides of the lesion
    cropped_mask = rotated_mask[y:y+h, x:x+w]
    
    return cropped_mask

# Example usage
if __name__ == "__main__":
    # Load the mask image
    mask_image = cv2.imread('PAT_492_1784_50_mask.png', cv2.IMREAD_GRAYSCALE)
    
    # Ensure mask image is loaded correctly
    if mask_image is None:
        print("Error: Failed to load the mask image.")
    else:
        # Find the longest diameter of the lesion
        longest_diameter = find_longest_diameter(mask_image)
        
        if longest_diameter is not None:
            # Crop the lesion image to the left and right sides divided by the longest diameter
            cropped_mask = crop_to_sides(mask_image, longest_diameter)
            
            # Display the cropped mask image
            cv2.imshow("Cropped Mask", cropped_mask)
            cv2.waitKey(0)
            cv2.destroyAllWindows()


## this code extend the background, then finds the longest diameter, and rotates it horizontally >180* or vertically, 
## then cropps to the minimum spanning box 

In [3]:
import os
import cv2
import numpy as np

def find_longest_diameter(mask_image):
    # Find contours of the lesion in the mask
    contours, _ = cv2.findContours(mask_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        print("Error: No contour found in the mask image.")
        return None
    
    # Find the convex hull of the largest contour
    largest_contour = max(contours, key=cv2.contourArea)
    hull = cv2.convexHull(largest_contour)
    
    # Find the longest diameter within the convex hull
    longest_diameter = find_longest_line(hull)
    
    return longest_diameter

def find_longest_line(contour):
    # Initialize variables to store the longest line and its length
    max_length = 0
    longest_line = None
    
    # Iterate through all combinations of points to find the longest line segment
    for i in range(len(contour)):
        for j in range(i + 1, len(contour)):
            # Calculate the length of the line segment
            length = np.linalg.norm(contour[i] - contour[j])
            
            # Update the longest line if the current line segment is longer
            if length > max_length:
                max_length = length
                longest_line = (contour[i][0], contour[j][0])
    
    return longest_line

def rotate_image(image, angle):
    # Get the dimensions of the image
    h, w = image.shape[:2]
    
    # Calculate the rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D((w/2, h/2), angle, 1)
    
    # Perform the rotation
    rotated_image = cv2.warpAffine(image, rotation_matrix, (w, h))
    
    return rotated_image

def crop_to_sides(mask_image, longest_diameter):
    # Determine the angle of rotation to make the longest diameter vertical
    angle = np.degrees(np.arctan2(longest_diameter[1][1] - longest_diameter[0][1], longest_diameter[1][0] - longest_diameter[0][0]))
    
    # If the angle is more than 90 degrees, subtract 180 degrees to make it negative
    if angle > 90:
        angle -= 180
    
    # Rotate the mask image
    rotated_mask = rotate_image(mask_image, angle)
    
    # Find contours of the rotated lesion
    contours, _ = cv2.findContours(rotated_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Find the bounding rectangle of the rotated lesion
    x, y, w, h = cv2.boundingRect(contours[0])
    
    # Crop the rotated mask image to include the left and right sides of the lesion
    cropped_mask = rotated_mask[y:y+h, x:x+w]
    
    return cropped_mask

if __name__ == "__main__":
    # Create a folder to save the cropped masks if it doesn't exist
    output_folder = "cropped&rotated_masks"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Iterate over all mask images in the current directory
    for filename in os.listdir():
        if filename.endswith("_mask.png"):
            # Load the mask image
            mask_image = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
            
            # Ensure mask image is loaded correctly
            if mask_image is None:
                print(f"Error: Failed to load the mask image {filename}.")
            else:
                # Find the longest diameter of the lesion
                longest_diameter = find_longest_diameter(mask_image)
                
                if longest_diameter is not None:
                    # Crop the lesion image to the left and right sides divided by the longest diameter
                    cropped_mask = crop_to_sides(mask_image, longest_diameter)
                    
                    # Save the cropped mask image
                    output_filename = os.path.join(output_folder, filename)
                    cv2.imwrite(output_filename, cropped_mask)


KeyboardInterrupt: 

In [14]:
#folding horizontally and vertically, see the mask (1 or 0 entry), checks how much both sides are the same (countNotZero)
#then calculates the fracton of similarity
# score 0: no similarity
#score 1: total similarity

import cv2

def calculate_pixel_differences(mask_path):
    # Load the mask image
    mask_image = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    
    if mask_image is None:
        print("Error: Failed to load the mask image.")
        return None
    
    # Total number of pixels in the mask image
    total_pixels = mask_image.size
    
    # Calculate the center of the mask
    center_x = mask_image.shape[1] // 2
    center_y = mask_image.shape[0] // 2
    
    # Split the mask into left and right halves
    left_half = mask_image[:, :center_x]
    right_half = mask_image[:, center_x:]
    
    # Calculate the pixel count differences on each half vertically
    left_pixel_count_vertical = cv2.countNonZero(left_half)
    right_pixel_count_vertical = cv2.countNonZero(right_half)
    vertical_difference = abs(left_pixel_count_vertical - right_pixel_count_vertical)
    
    # Split the mask into top and bottom halves
    top_half = mask_image[:center_y, :]
    bottom_half = mask_image[center_y:, :]
    
    # Calculate the pixel count differences on each half horizontally
    top_pixel_count_horizontal = cv2.countNonZero(top_half)
    bottom_pixel_count_horizontal = cv2.countNonZero(bottom_half)
    horizontal_difference = abs(top_pixel_count_horizontal - bottom_pixel_count_horizontal)
    
    # Calculate the fraction of similarity vertically
    vertical_similarity = 1 - (vertical_difference / total_pixels)
    
    # Calculate the fraction of similarity horizontally
    horizontal_similarity = 1 - (horizontal_difference / total_pixels)
    
    return vertical_similarity, horizontal_similarity

# Example usage
if __name__ == "__main__":
    mask_path = "cropped&rotated_masks/PAT_859_1638_598_mask.png"
    vertical_similarity, horizontal_similarity = calculate_pixel_differences(mask_path)
    print("Vertical Similarity:", vertical_similarity)
    print("Horizontal Similarity:", horizontal_similarity)


Vertical Similarity: 0.9023626975434205
Horizontal Similarity: 0.9052730402127993


In [39]:
##    0: Indicates complete dissimilarity or no similarity between the two sides.
##    This means that the pixel distribution on one side is entirely different from the other side.
##    1: Indicates complete similarity or perfect symmetry between the two sides. 
##    This means that the pixel distribution on both sides is exactly the same.

In [5]:
#get the scale for similarity:
# score 0: -no similarity-
#score 1:
#score 2:
#score 3: -total symetry -

In [15]:
def calculate_similarity_score(vertical_similarity, horizontal_similarity):
    if vertical_similarity > 0.95 and horizontal_similarity > 0.95:
        return 3
    elif vertical_similarity > 0.91 and horizontal_similarity > 0.91:
        return 2
    elif vertical_similarity > 0.8 and horizontal_similarity > 0.8:
        return 1
    else:
        return 0

# Example usage
if __name__ == "__main__":
    mask_path = "cropped&rotated_masks/PAT_859_1638_598_mask.png"
    vertical_similarity, horizontal_similarity = calculate_pixel_differences(mask_path)
    similarity_score = calculate_similarity_score(vertical_similarity, horizontal_similarity)
    print("Similarity Score:", similarity_score)

Similarity Score: 1


In [28]:
import cv2
import os
import csv

# Function to calculate pixel differences and return similarity score
def calculate_similarity_score(mask_path):
    # Load the mask image
    mask_image = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    
    if mask_image is None:
        print("Error: Failed to load the mask image:", mask_path)
        return None
    
    # Total number of pixels in the mask image
    total_pixels = mask_image.size
    
    # Calculate the center of the mask
    center_x = mask_image.shape[1] // 2
    center_y = mask_image.shape[0] // 2
    
    # Split the mask into left and right halves
    left_half = mask_image[:, :center_x]
    right_half = mask_image[:, center_x:]
    
    # Calculate the pixel count differences on each half vertically
    left_pixel_count_vertical = cv2.countNonZero(left_half)
    right_pixel_count_vertical = cv2.countNonZero(right_half)
    vertical_difference = abs(left_pixel_count_vertical - right_pixel_count_vertical)
    
    # Split the mask into top and bottom halves
    top_half = mask_image[:center_y, :]
    bottom_half = mask_image[center_y:, :]
    
    # Calculate the pixel count differences on each half horizontally
    top_pixel_count_horizontal = cv2.countNonZero(top_half)
    bottom_pixel_count_horizontal = cv2.countNonZero(bottom_half)
    horizontal_difference = abs(top_pixel_count_horizontal - bottom_pixel_count_horizontal)
    
    # Calculate the fraction of similarity vertically
    vertical_similarity = 1 - (vertical_difference / total_pixels)
    
    # Calculate the fraction of similarity horizontally
    horizontal_similarity = 1 - (horizontal_difference / total_pixels)
    
    # Calculate similarity score
    if vertical_similarity > 0.98 and horizontal_similarity > 0.98:
        return 3
    elif vertical_similarity > 0.95 and horizontal_similarity > 0.95:
        return 2
    elif vertical_similarity > 0.85 and horizontal_similarity > 0.85:
        return 1
    else:
        return 0

# Path to the folder containing masks
masks_folder_path = 'cropped&rotated_masks'

# Open the CSV file for reading
with open('imageID_sheet.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip header row

    # Open a new CSV file for writing
    with open('modified_imageID_sheet.csv', 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        
        # Write header row
        writer.writerow(["ImageID", "Similarity Score"])
        
        # Iterate over rows in the CSV file
        for row in reader:
            image_id = row[0]
            mask_name_1 = image_id + '_mask.png'
            mask_name_2 = image_id + '_mask_mask.png'
            
            mask_path_1 = os.path.join(masks_folder_path, mask_name_1)
            mask_path_2 = os.path.join(masks_folder_path, mask_name_2)
            
            # Check if the mask exists with either name
            if os.path.exists(mask_path_1):
                mask_path = mask_path_1
            elif os.path.exists(mask_path_2):
                mask_path = mask_path_2
            else:
                print("Error: Mask not found for image ID:", image_id)
                continue

            # Calculate similarity score
            similarity_score = calculate_similarity_score(mask_path)

            # Write the image ID and similarity score to the output CSV file
            writer.writerow([image_id, similarity_score])

print("Done")


Error: Mask not found for image ID: PAT_771_1489_345
Error: Mask not found for image ID: PAT_153_233_45
Error: Mask not found for image ID: PAT_246_377_159
Error: Mask not found for image ID: PAT_356_4511_960
Error: Mask not found for image ID: PAT_1618_2771_628
Done
